# YOLO

In [1]:
import collections

import chainer
from chainer import functions as F
from chainer import initializers
from chainer import links as L

## モデル

YOLOは畳み込みニューラルネットワークを用いた高速な物体検出アルゴリズムである。  
(YOLOにはv1, v2, v3があるが、YOLO v1の)ネットワーク全体の構造は以下のとおりである。

<img src="image/yolo.png", style="width: 600px;">  
(引用 「You Only Look Once: Unified, Real-Time Object Detection」)

YOLOでは、画像を固定グリッド（$7 \times 7$）に分割し、各セルごとに（2個の）バウンディングボックスの位置・大きさと信頼度、  
そしてその中の物体クラス（20個の物体）の確率を出力する。

In [2]:
class YOLO(chainer.Chain):
    # YOLO v1
    def  __init__(self):
        super(YOLO, self).__init__()
        # 重みの初期値
        kwargs = {
            'initialW': initializers.Normal(0.01),
            'initial_bias': initializers.Zero(),
            }
        
        with self.init_scope():
            self.conv1 = L.Convolution2D(3, 64, ksize=7, stride=2, pad=3, **kwargs)
            
            self.conv2 = L.Convolution2D(64, 192, ksize=3, stride=1, pad=1, **kwargs)
            
            self.conv3_1 = L.Convolution2D(192, 128, ksize=1, stride=1, pad=0, **kwargs)
            self.conv3_2 = L.Convolution2D(128, 256, ksize=3, stride=1, pad=1, **kwargs)
            self.conv3_3 = L.Convolution2D(256, 256, ksize=1, stride=1, pad=0, **kwargs)
            self.conv3_4 = L.Convolution2D(256, 512, ksize=3, stride=1, pad=1, **kwargs)
            
            self.conv4_1 = L.Convolution2D(512, 256, ksize=1, stride=1, pad=0, **kwargs)
            self.conv4_2 = L.Convolution2D(256, 512, ksize=3, stride=1, pad=1, **kwargs)
            self.conv4_3 = L.Convolution2D(512, 256, ksize=1, stride=1, pad=0, **kwargs)
            self.conv4_4 = L.Convolution2D(256, 512, ksize=3, stride=1, pad=1, **kwargs)
            self.conv4_5 = L.Convolution2D(512, 256, ksize=1, stride=1, pad=0, **kwargs)
            self.conv4_6 = L.Convolution2D(256, 512, ksize=3, stride=1, pad=1, **kwargs)
            self.conv4_7 = L.Convolution2D(512, 256, ksize=1, stride=1, pad=0, **kwargs)
            self.conv4_8 = L.Convolution2D(256, 512, ksize=3, stride=1, pad=1, **kwargs)
            self.conv4_9 = L.Convolution2D(512, 512, ksize=1, stride=1, pad=0, **kwargs)
            self.conv4_10 = L.Convolution2D(512, 1024, ksize=3, stride=1, pad=1, **kwargs)
            
            self.conv5_1 = L.Convolution2D(1024, 512, ksize=1, stride=1, pad=0, **kwargs)
            self.conv5_2 = L.Convolution2D(512, 1024, ksize=3, stride=1, pad=1, **kwargs)
            self.conv5_3 = L.Convolution2D(1024, 512, ksize=1, stride=1, pad=0, **kwargs)
            self.conv5_4 = L.Convolution2D(512, 1024, ksize=3, stride=1, pad=1, **kwargs)
            self.conv5_5 = L.Convolution2D(1024, 1024, ksize=3, stride=1, pad=1, **kwargs)
            self.conv5_6 = L.Convolution2D(1024, 1024, ksize=3, stride=2, pad=1, **kwargs)
            
            self.conv6_1 = L.Convolution2D(1024, 1024, ksize=3, stride=1, pad=1, **kwargs)
            self.conv6_2 = L.Convolution2D(1024, 1024, ksize=3, stride=1, pad=1, **kwargs)
                        
            self.fc6 = L.Linear(1024 * 7 * 7, 512, **kwargs)
            self.fc7 = L.Linear(512, 4096, **kwargs)
            self.fc8 = L.Linear(4096, 7 * 7 * 30, **kwargs)
            
            """
            output shape: 7*7*30 = (7, 7, 30)   (7×7のグリッドそれぞれに対して30次元)
            (7, 7, 20) : (20個の)物体クラスの確率
            (7, 7, 2) : (2個の)バウンディングの信頼度
            (7, 7, 2*4) : (2個の)バウンディングの位置・大きさ (4次元 x, y, w, h)
            """
        
        self.functions = self.orderd_functions()
    
    def __call__(self, x):
        h = x
        for key, funcs in self.functions.items():
            for func in funcs:
                h = func(h)
        return h
    
    def orderd_functions(self):
        return collections.OrderedDict([               # size: 448
            ('conv1',    [self.conv1, _leaky_relu]),      # size: 224
            ('pool1',     [_max_pooling_2d]),             # size: 112
            
            ('conv2',    [self.conv2, _leaky_relu]),
            ('pool2',     [_max_pooling_2d]),              # size: 56
            
            ('conv3_1', [self.conv3_1, _leaky_relu]),
            ('conv3_2', [self.conv3_2, _leaky_relu]),
            ('conv3_3', [self.conv3_3, _leaky_relu]),
            ('conv3_4', [self.conv3_4, _leaky_relu]),
            ('pool3',     [_max_pooling_2d]),              # size: 28
            
            ('conv4_1', [self.conv4_1, _leaky_relu]),
            ('conv4_2', [self.conv4_2, _leaky_relu]),
            ('conv4_3', [self.conv4_3, _leaky_relu]),
            ('conv4_4', [self.conv4_4, _leaky_relu]),
            ('conv4_5', [self.conv4_5, _leaky_relu]),
            ('conv4_6', [self.conv4_6, _leaky_relu]),
            ('conv4_7', [self.conv4_7, _leaky_relu]),
            ('conv4_8', [self.conv4_8, _leaky_relu]),
            ('conv4_9', [self.conv4_9, _leaky_relu]),
            ('conv4_10', [self.conv4_10, _leaky_relu]),      
            ('pool4',       [_max_pooling_2d]),            # size: 14
            
            ('conv5_1', [self.conv5_1, _leaky_relu]),
            ('conv5_2', [self.conv5_2, _leaky_relu]),
            ('conv5_3', [self.conv5_3, _leaky_relu]),
            ('conv5_4', [self.conv5_4, _leaky_relu]),
            ('conv5_5', [self.conv5_5, _leaky_relu]),
            ('conv5_6', [self.conv5_6, _leaky_relu]),   # size: 7
            
            ('fc6',        [self.fc6, _leaky_relu]),
            ('fc7',        [self.fc7, _leaky_relu, F.dropout]),
            ('fc8',        [self.fc8])
        ])
    
def _max_pooling_2d(x):
    return F.max_pooling_2d(x, ksize=2)

def _leaky_relu(x):
    return F.leaky_relu(x, slope=0.1)

In [3]:
yolo = YOLO()

In [4]:
import numpy as np
img = np.ones((1, 3, 448, 448), dtype=np.float32)
output = yolo(img)
print(output.shape)

(1, 1470)


## 損失関数

<img src="image/yolo_loss.png", style="width: 500px;">  

$\lambda_{\text{coord}}$: バウンディングボックスの位置に関するロスの大きさを制御するパラメータ  
$\lambda_{\text{noobj}}$: 物体の含まれていないバウンディングボックスに対するロスの大きさを制御するパラメータ  
$S^{2}$: グリッド数  
$B$: バウンディングボックスの数  
$\mathbb{1}_{ij}^{obj}$: セル$i$のバウンディングボックス$j$に物体が存在するとき$1$、存在しないとき$0$となる  
$\mathbb{1}_{ij}^{noobj}$: セル$i$のバウンディングボックス$j$に物体が存在しないとき$1$、存在するとき$0$となる  
$x_{i}, y_{i}, w_{i}, h_{i}$: セル$i$における予測したバウンディングボックスの座標と幅、高さ  
$\hat{x}_{i}, \hat{y}_{i}, \hat{w}_{i}, \hat{h}_{i}$: セル$i$における真のバウンディングボックスの位置・大きさ  
$C_{i}$: セル$i$における予測したバウンディングボックスの信頼度  
$\hat{C}_{i}$: セル$i$における真のバウンディングボックスの信頼度  
$p_{i}(c)$: セル$i$のにおける予測した物体クラス$c$の確率  
$\hat{p}_{i}(c)$: セル$i$のにおける真の物体クラス$c$の確率

In [5]:
def yolo_loss(obj, noobj, x_prob, x_conf, x_loc_xy, x_loc_wh, 
              t_prob, t_conf, t_loc_xy, t_loc_wh, lam_obj, lam_noobj):
    # objは物体があるインデックスがTrueのarray (noobjも同様)
    
    loss_loc_xy = (x_loc_xy - t_loc_xy)**2
    loss_loc_wh = (np.sqrt(x_loc_wh) - np.sqrt(t_loc_wh))**2
    loss_loc = (obj * (loss_loc_xy + loss_loc_wh)).sum()
    
    loss_c_obj = (obj * (x_conf - t_conf)**2).sum()
    loss_c_noobj = (noobj * (x_conf - t_conf)**2).sum()
    loss_p = (obj * (x_prob - t_prob)**2).sum()
    
    loss = lam_obj*loss_loc + loss_c_obj + lam_noobj*loss_c_noobj + loss_p
    return loss